# Reddit Data Collector

In [8]:
import requests
import pandas as pd
from datetime import datetime
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
#pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)

In [45]:
# we use this function to convert responses to dataframes
global post_id_array 
post_id_array = []
global post_link_array 
post_link_array = []
def df_from_response(res):
    global post_id_array
    global post_link_array
    # initialize temp dataframe for batch of data in response
    df = pd.DataFrame()

    # loop through each post pulled from res and append to df
    for post in res.json()['data']['children']:
        df = df.append({
            'subreddit': post['data']['subreddit'],
            'title': post['data']['title'],
            'selftext': post['data']['selftext'],
            'upvote_ratio': post['data']['upvote_ratio'],
            'ups': post['data']['ups'],
            'downs': post['data']['downs'],
            'score': post['data']['score'],
            'link_flair_css_class': post['data']['link_flair_css_class'],
            'created_utc': datetime.fromtimestamp(post['data']['created_utc']).strftime('%Y-%m-%dT%H:%M:%SZ'),
            'id': post['data']['id'],
            'kind': post['kind']
        }, ignore_index=True)

        #post_id_array.append(post['data']['id'])
        post_link_array.append(post['data']['permalink'])
        #print(post['data'])
    return df


# authenticate API
client_auth = requests.auth.HTTPBasicAuth(' ', ' ')
data = {
    'grant_type': 'password',
    'username': ' ',
    'password': ' '
}
headers = {'User-Agent': 'myBot/0.0.1'}

# send authentication request for OAuth token
res = requests.post('https://www.reddit.com/api/v1/access_token',
                    auth=client_auth, data=data, headers=headers)
# extract token from response and format correctly
TOKEN = f"bearer {res.json()['access_token']}"
# update API headers with authorization (bearer token)
headers = {**headers, **{'Authorization': TOKEN}}

# initialize dataframe and parameters for pulling data in loop
data = pd.DataFrame()
params = {'limit': 100}

# loop through 10 times (returning 1K posts)
for i in range(3):
    # make request
    #https://www.reddit.com/search/?q=elon%20musk%20twitter&sort=top&t=week
    #"https://oauth.reddit.com/r/politics/top/?t=week"
    #"https://oauth.reddit.com/search/?q=elon%20musk%20twitter&sort=top&t=week"
    #https://www.reddit.com/r/technology/comments/ubjp7g/twitter_to_accept_elon_musks_45_billion_bid_to/
    res = requests.get("https://oauth.reddit.com/search/?q=elon%20musk%20twitter&sort=top&t=week",headers=headers,params=params)

    # get dataframe from response
    new_df = df_from_response(res)
    # take the final row (oldest entry)
    row = new_df.iloc[len(new_df)-1]
    # create fullname
    fullname = row['kind'] + '_' + row['id']
    # add/update fullname in params
    params['after'] = fullname
    
    # append new_df to data
    data = data.append(new_df, ignore_index=True)
    
print(post_link_array)

['/r/technology/comments/ubjp7g/twitter_to_accept_elon_musks_45_billion_bid_to/', '/r/technology/comments/ucqidk/tesla_shares_sink_wipe_out_over_125_billion_in/', '/r/HolUp/comments/ubejhb/elon_musk_shitposting_jesus/', '/r/technology/comments/uej4db/elon_musk_suggested_firing_workers_and_working/', '/r/fakehistoryporn/comments/ueh22b/elon_musk_after_implanting_the_first_neuralink/', '/r/news/comments/ubjass/twitter_set_to_accept_best_and_final_offer_of/', '/r/worldnews/comments/ud4qay/eu_warns_elon_musk_that_being_too_lax_on_twitter/', '/r/wallstreetbets/comments/u9wzcr/elon_musk_confirms_that_bill_gates_has_a_500/', '/r/technology/comments/uccz0i/eu_warns_elon_musk_over_twitter_moderation_plans/', '/r/technology/comments/ub4gtq/twitter_is_reportedly_reconsidering_elon_musks/', '/r/wallstreetbets/comments/uce05o/elons_first_day_at_twitter_hq/', '/r/technology/comments/ubvwld/elon_musk_pledges_to_authenticate_all_humans_as/', '/r/technews/comments/ubshnj/twitter_accepts_buyout_giving_e

In [10]:
with pd.option_context('display.max_rows', 10, 'display.max_columns', None): 
    display(data)

,subreddit,title,selftext,upvote_ratio,ups,downs,score,link_flair_css_class,created_utc,id,kind
0,technology,Twitter to accept Elon Musk’s $45 billion bid ...,,0.82,62945,0,62945,general,2022-04-25T08:11:59Z,ubjp7g,t3
1,technology,"Tesla shares sink, wipe out over $125 billion ...",,0.81,59689,0,59689,general,2022-04-26T20:07:31Z,ucqidk,t3
2,HolUp,Elon Musk shitposting Jesus,,0.83,53732,0,53732,,2022-04-25T02:26:18Z,ubejhb,t3
3,technology,Elon Musk suggested firing workers and working...,,0.82,40266,0,40266,general,2022-04-29T06:45:23Z,uej4db,t3
4,fakehistoryporn,Elon musk after implanting the first Neuralink...,,0.93,37867,0,37867,2015,2022-04-29T04:08:03Z,ueh22b,t3
...,...,...,...,...,...,...,...,...,...,...,...
242,EnoughPCMSpam,PCM goes crazy over Elon Musk buying Twitter (...,,0.98,566,0,566,,2022-04-25T20:26:02Z,ubzryg,t3
243,GenUsa,BASED ELON MUSK 🇺🇸🇺🇸🇺🇸🇺🇸 AMERICA#1,,0.94,563,0,563,,2022-04-25T19:49:22Z,ubz1ck,t3
244,Shitstatistssay,Psaki expresses Biden's desire to ‘Reform’ sec...,,0.96,555,0,555,None,2022-04-26T20:34:07Z,ucr0ek,t3
245,conspiracy,"Elon Musk takes over Twitter, and look at what...",,0.89,559,0,559,None,2022-04-27T10:31:04Z,ud4xwi,t3


In [32]:
data.to_csv('saved_files/reddit_elonmusktwitter2.csv', encoding='utf-8', index=False)

In [131]:
import praw
from urllib.parse import urlparse
import re

num_comments = 20
i = 0
reddit = praw.Reddit(client_id=' ', 
                     client_secret=' ',
                     user_agent='myBot/0.0.1')

df2 = pd.DataFrame(columns=["comment", "comment_score"])

comment_array = []
for post_link in post_link_array:
    url = 'https://www.reddit.com'+ post_link
    #print(url)
    #print(" ")
    _, listing = reddit.get(urlparse(url).path, params=dict(sort='top', limit=100000000))
                            
    for i in range(num_comments+1):
        try:
            comment = listing.children[i]
            #print("forloop #: ",i)
            #print(comment.score, comment.body)
            #this_column = df2.columns[i]
            #print(comment.body)
            comment_clean = re.sub(r'[^A-Za-z0-9 ]+', '', comment.body)
            #print(comment_clean)
            comment_array.append([i,comment_clean, comment.score])
        except:
            comment_array.append([i,'', ''])
    
    

#print(comment_array)
df2 = pd.DataFrame(comment_array,columns=['Comment #', 'Text','Upvotes'])

In [132]:
with pd.option_context('display.max_rows', 10, 'display.max_columns', None): 
    display(df2)

,Comment #,Text,Upvotes
0,0,FORTY FIVE BILLION jesus well I guess Im rich ...,13079
1,1,Between Netflix making crazy choices and now t...,8763
2,2,I hope he buys it just for it to become the My...,6838
3,3,removed,5503
4,4,removed,5411
...,...,...,...
5182,16,The one time I caught a suspension I was talki...,2
5183,17,I want to see blackfacehitler trend again,2
5184,18,If thats whats trending thinking it is trendin...,1
5185,19,Financial Markets giant trending during one of...,1


In [133]:
df2.to_csv('saved_files/reddit_elonmusktwitter_top20comments_clean.csv', encoding='utf-8', index=False)